In [ ]:
import scanpy as sc
import anndata as ann
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors

import os 
#doublet detection
import scrublet as scr
#batch correction (not installed by default in the docker container, install via command line: pip install scanorama bbknn)
import scanorama as scan
#external modules
import scanpy.external as sce
#pretty plotting
import seaborn as sb

#R interface
import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri


In [ ]:
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()


Of note, this notebook was created as part of a workshop, so we use extra large legend texts in all seaborn plots. You can set the context as well to 'talk' or 'paper'.

In [ ]:
sb.set_context(context='poster')


In [ ]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
#.libPaths(.libPaths()[c(3,2,1)])

# Load all the R libraries we will be using in the notebook
library(scran)
library(RColorBrewer)
library(DropletUtils)


## Set project file paths

We set up the file paths to the respective directories.

In [ ]:
file_path = '/root/host_home/Documents/ICB/Teaching/2007_scanpy_HMGU/' #this is my file path please adapt it to your directory

File path to the raw data. They are usually stored at a different location than the rest of the project.

In [ ]:
file_path_raw = file_path + '3k_PBMC/'

The data directory contains all processed data and `anndata` files. 

In [ ]:
data_dir = file_path + 'day1_beginner/data/' 

The tables directory contains all tabular data output, e.g. in `.csv` or `.xls` file format. That applies to differential expression test results or overview tables such as the number of cells per cell type.

In [ ]:
table_dir = file_path + 'day1_beginner/tables/'

The default figure path is a POSIX path calles 'figures'. If you don't change the default figure directory, scanpy creates a subdirectory where this notebook is located.  

In [ ]:
sc.settings.figdir = file_path + 'day1_beginner/figures/'

**Comment:** When you repeat certain analyses, it might be helpful to set a `date` variable and add it to every figure and table (see `datetime` Python package).

# Read data

The dataset consists of 4k PBMCs (Human) provided by 10X Genomics. The data is an mtx directory with an `mtx` file (*i.e.* count matrix), two `tsv` files with barcodes (*i.e.* cell indices) and features (*i.e.* gene symbols). `Scanpy` unpacks the files (if the files are in `gz` archive format) and creates an `anndata` object with the `read_10x_mtx` function.    

The dataset is not filtered, yet. 

In [ ]:
file_path_raw = file_path_raw + 'raw_gene_bc_matrices/'


In [ ]:
adata_raw = sc.read_10x_mtx(path=file_path_raw)

Let us check the dataset size. 

In [ ]:
adata_raw.shape

In [ ]:
print('Total number of observations: {:d}'.format(adata_raw.n_obs))

**Comment:** End of first session.

# Pre-processing and visualization

## Remove empty droplets

The dataset contains an excessive amount of "cells", which are in fact empty droplets. Let us remove these barcodes prior to further quality control. We use emptyDrops to compute if a cell is a cell or an empty droplet.

It must be noted that CellRanger 3.0 has incorporated the EmptyDrops algorithm to distinguish cells from empty droplets.

Prepare input for EmptyDrops.

In [ ]:
sparse_mat = adata_raw.X.T
genes = adata_raw.var_names
barcodes = adata_raw.obs_names

Run EmptyDrops.

In [ ]:
%%R -i sparse_mat -i genes -i barcodes -o barcodes_filtered -o ambient_genes

sce <- SingleCellExperiment(assays = list(counts = sparse_mat), colData=barcodes)
rownames(sce) <- genes 
ambient <- emptyDrops(counts(sce))
is_cell <- ambient$FDR <= 0.05
threshold_ambient <- 0.005
ambient_genes <- names(ambient@metadata$ambient[ambient@metadata$ambient> threshold_ambient,])
barcodes_filtered <- barcodes[which(is_cell)]

Empty drops returns a list of potentially ambient genes and the barcodes, which belong to actual cells.

In [ ]:
ambient_genes

In [ ]:
barcodes_filtered

Let us create a filtered data matrix using the filtered barcodes.

In [ ]:
adata = adata_raw[np.in1d(adata_raw.obs_names, barcodes_filtered)].copy()

In [ ]:
adata

**BONUS**: Examine the level of background gene expression.

Save the filtered data set to file.

In [ ]:
adata.write(data_dir + 'data_filtered.h5ad')

## Quality control

Read data from file to begin with the quality control.

In [ ]:
adata = sc.read(data_dir + 'data_filtered.h5ad')

Data quality control can be split into cell QC and gene QC. Typical quality measures for assessing the quality of a cell include the number of molecule counts (UMIs), the number of expressed genes, and the fraction of counts that are mitochondrial. A high fraction of mitochondrial reads being picked up can indicate cell stress, as there is a low proportion of nuclear mRNA in the cell. It should be noted that high mitochondrial RNA fractions can also be biological signals indicating elevated respiration.

We start with calculating the QC covariates: 

* total number of counts per cell
* number of expressed genes per cell
* fraction of mitochondrial reads per cell

**Task:** Compute the library size (i.e. the total number of counts per cell), compute the log-scaled library size and the number of expressed genes per cell.

**BONUS:** Compute the mean gene expression and the fraction of cells that express a gene. Compute the mean gene expression per batch, donor and replicate.

In [ ]:
# Quality control - calculate QC covariates
adata.obs['n_counts'] = #
adata.obs['log_counts'] = #
adata.obs['n_genes'] = #

Compute the fraction of mitochondrial genes. Note: mitochondrial genes in human start with 'MT-'.

In [ ]:
mt_gene_mask = np.flatnonzero([gene.startswith('MT-') for gene in adata.var_names])
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
adata.obs['mt_frac'] = np.sum(adata[:, mt_gene_mask].X, axis=1).A1/adata.obs['n_counts']

Let us visualize the number of expressed genes and the number of counts as a scatter plot. 

**Task:** Create a scatter plot with the library size against the number of genes. Create a second plot, where you only show cells with a library size of less than 10,000 counts. Color by the fraction of mitochondrial reads. 

**Questions:** How can we describe the relation of library size vs number of expressed genes vs mitochondrial reads?

**BONUS:** Plot the highest expressed genes. 



In [ ]:
#Data quality summary plots
p1 = sc.pl.scatter(adata = , x='' , y='', color='', size=40)
#hint: temporary subsetting of the anndata object by .obs works like adata[adata.obs['key']<value]
p2 = sc.pl.scatter(adata = , x='', y='', 
                   color='', size=40)

**Task:** Below, you find a the code to create a violin plot of the library size. Create another two violin plots displaying the number of genes and the fraction of mitochondrial reads.

**Questions:** How do the count data distribute within the sample? 


In [ ]:
#Sample quality plots
rcParams['figure.figsize']=(7,7)
t1 = sc.pl.violin(adata, 'n_counts',
                  #groupby='sample',
                  size=2, log=True, cut=0)
t1 = sc.pl.violin() #display number of genes
t2 = sc.pl.violin() #display the fraction of mitochondrial reads

**Conclusions:** By looking at plots of the number of genes versus the number of counts with MT fraction information, we can assess whether there are cells with unexpected summary statistics. It is important here to look at these statistics jointly.  We should probably still filter out some cells with very few genes as these may be difficult to annotate later. This will be true for the initial cellular density between 1000-4000 counts and < ~500 genes.

Furthermore it can be seen in the main cloud of data points, that cells with lower counts and genes tend to have a higher fraction of mitochondrial counts. These cells are likely under stress or are dying. When apoptotic cells are sequenced, there is less mRNA to be captured in the nucleus, and therefore fewer counts overall, and thus a higher fraction of counts fall upon mitochondrial RNA. If cells with high mitochondrial activity were found at higher counts/genes per cell, this would indicate biologically relevant mitochondrial activity.

**Task:** Create a histogram for the total number of counts. Further, create a histogram for the low count and high count regime, each. Here, you have to decide on a reasonable threshold. 

Note: `pandas` does some histogram plotting with `adata.obs['n_counts'].hist()`, howecer, you will obtain prettier plots with `distplot` from `seaborn`.    

In [ ]:
#Thresholding decision: counts
rcParams['figure.figsize']=(20,5)
fig_ind=np.arange(131, 134)
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.6)

p3 = sb.distplot(adata.obs['n_counts'], 
                 kde=False, 
                 ax=fig.add_subplot(fig_ind[0]))
p4 = sb.distplot( #histogram for low count regime
                 ax=fig.add_subplot(fig_ind[1]))
p5 = sb.distplot( #histogram for high count regime
                 ax=fig.add_subplot(fig_ind[2]))
plt.show()

**Conclusions:**
Histograms of the number of counts per cell show a small peak of groups of cells with fewer than **XX** counts, which are likely uninformative given the overall distribution of counts. This may be cellular debris found in droplets.

On the upper end of counts, we see a population of cells with high counts with decaying slope at **XX** counts. We estimate this population to range until **XX** counts. This estimation is performed by visually tracing a Gaussian around the population.

**Task:** Create a histogram for the total number of genes. Further, create a histogram for the low gene count regime.

In [ ]:
#Thresholding decision: genes

rcParams['figure.figsize']=(20,5)
fig_ind=np.arange(131, 133)
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.6) #create a grid for subplots

p6 = sb.distplot(adata.obs['n_genes'], kde=False, bins=60, ax=fig.add_subplot(fig_ind[0]))
#low number of genes regime
p7 = sb.distplot( ax=fig.add_subplot(fig_ind[1]))
plt.show()

**Conclusions:**
Two populations of cells with low gene counts can be seen in the above plots. Given these plots, and the plot of genes vs counts above, we decide to filter out cells with fewer than **XX** genes expressed. Below this we are likely to find dying cells or empty droplets with ambient RNA. Looking above at the joint plots, we see that we filter out the main density of low gene cells with this threshold.

In general it is a good idea to be permissive in the early filtering steps, and then come back to filter out more stringently when a clear picture is available of what would be filtered out. This is available after visualization/clustering. For demonstration purposes we stick to a simple (and slightly more stringent) filtering here.

**Task:** Create a histogram for the fraction of mitochondrial genes. Further, create a histogram for the high fraction regime.

In [ ]:
#Thresholding decision: mitochondrial reads

rcParams['figure.figsize']=(20,5)
fig_ind=np.arange(131, 133)
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.6)

p8 = sb.distplot(  #display the fraction of mitochondrial reads
                 ax=fig.add_subplot(fig_ind[0]))

p9 = sb.distplot(  #display the fraction of mitochondrial reads for the high fraction (in this case a threshold of 0.2 as high)
                 ax=fig.add_subplot(fig_ind[1]))
plt.show()

**Task:** Filter your cells according for the total number of counts, number of expressed genes and fraction of mitochondrial reads. Check the number of remaining cells after each filtering step.

In [ ]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = )
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, max_counts = )
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

adata = adata[adata.obs['mt_frac'] < ]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = )
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))

**Task:** Next, filter out non-expressed genes. Check the number of remaining genes after filtering.

In [ ]:
#Filter genes:
print('Total number of genes: {:d}'.format(adata.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, )

print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

The filtering is performed based on the thresholds we identified from the QC plots. Genes are also filtered if they are not detected in at least **XX** cells. This reduces the dimensions of the matrix by removing 0 count genes and genes which are not sufficiently informative of the dataset.

### Doublet score

Let us estimate the amount of doublets in the dataset. Here, we use the tool `scrublet` that simulates doublet gene expression profiles based on the data. We apply it for each sample separately.

In [ ]:
adata.obs['doublet_score']= np.zeros(adata.shape[0])
adata.obs['doublet'] = np.zeros(adata.shape[0])

In [ ]:
# filtering/preprocessing parameters:
min_counts = 2
min_cells = 3
vscore_percentile = 85
n_pc = 50

# doublet detector parameters:
expected_doublet_rate = 0.02 
sim_doublet_ratio = 3
n_neighbors = 15



scrub = scr.Scrublet(counts_matrix = adata.X,  
                     n_neighbors = n_neighbors,
                     sim_doublet_ratio = sim_doublet_ratio,
                     expected_doublet_rate = expected_doublet_rate)

doublet_scores, predicted_doublets = scrub.scrub_doublets( 
                    min_counts = min_counts, 
                    min_cells = min_cells, 
                    n_prin_comps = n_pc,
                    use_approx_neighbors = True, 
                    get_doublet_neighbor_parents = False)

adata.obs['doublet_score'] = doublet_scores
adata.obs['doublet'] = predicted_doublets


**Tasks:** Plot the doublet score as a histogram and as violin plot. 

In [ ]:
rcParams['figure.figsize']=(6,6)
sb.distplot() #histogram of the doublet score
plt.show()

rcParams['figure.figsize']=(15,7)
sc.pl.violin() #violin plot of the doublet score

### filtering doublets

Scrublet proposed a different threshold than we would choose based upon the histogram plot of the doublet scores.

**Tasks:** Decide on a threshold to filter doublets. 

In [ ]:
thr = #add threshold
ix_filt = adata.obs['doublet_score']<=thr

adata = adata[ix_filt].copy()
print('Number of cells after doublet filter: {:d}'.format(adata.n_obs))

### Summarize sample information

In order to group by `batch` (for future purposes, because we presently deal with one sample), let us add a `batch` covariate to the `adata` object. 

In [ ]:
adata.obs['batch'] = '1'

df = adata.obs[['n_genes','n_counts', 'batch']]
df_all = pd.DataFrame(df.groupby(by='batch')['n_genes'].apply(np.mean).values,
                      index=df.groupby(by='batch')['n_genes'].apply(np.mean).index,
                      columns=['mean_genes'])

df_all['median_genes']=df.groupby(by='batch')['n_genes'].apply(np.median).values
df_all['mean_counts']=df.groupby(by='batch')['n_counts'].apply(np.mean).values
df_all['median_counts']=df.groupby(by='batch')['n_counts'].apply(np.median).values
df_all

Display table.

In [ ]:
df_all

**Tasks:** Save the summary table to file (`csv` or `xlsx` format) to the `tables` subdirectory. 

**Comment:** End of second session.

## Normalization

So far, our dataset is a count matrix. Here, every count corresponds to an mRNA molecule captured in the scRNA-seq experiment. As not all mRNA molecules in a cell are captured, there is a variability in the total number of counts detected between cells that results from both the number of molecules that were in the cells, and the sampling. As we cannot assume that all cells contain an equal number of molecules (cell sizes can differ substantially), we have to estimate the number of molecules that were initially in the cells. In fact, we don't estimate the exact number of molecules, but instead estimate cell-specific factors that should be proportional to the true number of molecules. These are called size factors. Normalized expression values are calculated by dividing the measured counts by the size factor for the cell.

Several methods for normalization for scRNA-seq data have been proposed. Here, we the `scran` library size normalization followed by log-transformation. In addition, we save the count matrix to `layers` as 'counts'.

**Comment:** We have (not comprehensively) tested whether normalisation per sample or all samples jointly gives more accurate results. When we normalised per sample, observed many more differentially expressed genes across conditions (in the range of thousands). We think that normalisation per sample preserves a systematic bias while joint normalisation removes batch effects within the same cluster partially, if a cluster contains cell from several batches. No such effect of the normalisation can be observed when samples do not overlap at all. For the time being, we perform joint normalisation of all samples. 

In [ ]:
#Perform a clustering for scran normalization in clusters
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)
sc.tl.louvain(adata_pp, key_added='groups', resolution=0.5)

In [ ]:
#Preprocess variables for scran normalization
input_groups = adata_pp.obs['groups']
data_mat = adata.X.T

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = computeSumFactors(data_mat, clusters=input_groups, min.mean=0.1)

**Tasks:** Save the size factors to the both anndata objects. Create a scatter plot of size factor vs library size and number of expressed genes, respectively. Repeat with `adata_pp` and color by louvain cluster (covariate `groups`). Create a histogram over all size factors.

**Questions:** How does the size factor relate to library size and number of expressed genes? Can we observe differences for the louvain clusters?


In [ ]:
# Visualize the estimated size factors
adata.obs['size_factors'] = 
adata_pp.obs['size_factors'] = 

rcParams['figure.figsize']=(8,8)
sc.pl.scatter() #color by library size
sc.pl.scatter() #color by number of expressed genes

#let us visualise how size factors differ across clusters
sc.pl.scatter() #color by library size
sc.pl.scatter() #color by number of expressed genes


sb.distplot() #histogram of size factors
plt.show()

Save the count matrix as layer. 

In [ ]:
adata.layers['counts'] = adata.X.copy()

Normalize with scran size-factors and log-scale. 

In [ ]:
adata.X /= adata.obs['size_factors'].values[:,None]
sc.pp.log1p(adata)

Modify the format of the resulting data matrix.

In [ ]:
adata.X = np.asarray(adata.X)

Free memory.

In [ ]:
del adata_pp

## Batch correction - general remarks

This dataset consists of a single batch, thus, batch effect correction is not an issue here. If you handle several batches, you may observe differences across samples, for instance, in the library size per dataset. Such differences may contribute to the batch effect. [Büttner et al., Nat Meth (2019)](https://www.nature.com/articles/s41592-018-0254-1) compared the performance of several batch correction methods. For low-to-medium complexity datasets, ComBat performed best among the tested tools. ComBat is also available in `scanpy` (see `sc.pp.combat`). 

For high complexity data, especially when you encounter changes in cell type composition, consider to use a data integration method of your choice. We distinguish three different types, *i.e.* if the methods create a corrected data matrix (in feature space), an embedding or a knn-graph. Examples are:
1. feature space: MNN (`scanpy.external.pp.mnn_correct`), scanorama (integrates with scanpy), Seurat v3 (R based)
2. embedding: scVI (Python based), Harmony (R based), scanorama
3. knn-graph: conos (R based), BBKNN (`scanpy.external.pp.bbknn`)

Several benchmarking studies aimed to determine best performing methods. In simple cases, Seurat v3 and Harmony performed best [Tran et al., Genome Biology (2020)](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1850-9) and [Chazarra-Gil et al., biorxiv (2020)](https://www.biorxiv.org/content/10.1101/2020.05.22.111211v2). More complex scenarios have been benchmarked in [Luecken et al., biorxiv (2020)](https://www.biorxiv.org/content/10.1101/2020.05.22.111161v2), where BBKNN, Scanorama, and scVI performed well. Furthermore, Scanorama had high scores in the preservation of biological signals, while BBKNN tended to overcorrect. Moreover, Luecken et al. tested different pre-processing schemes. In general, **selecting highly variable genes prior to batch correction improved the batch effect correction result**. We continue with the selection of highly variable genes. 


## Feature selection (Highly variable genes)

We extract highly variable genes (HVGs) to further reduce the dimensionality of the dataset and include only the most informative genes. Genes that vary substantially across the dataset are informative of the underlying biological variation in the data. As we only want to capture biological variation in these genes, we select highly variable genes after normalization and batch correction. HVGs are used for clustering, trajectory inference, and dimensionality reduction/visualization, while the full data set is used for computing marker genes, differential testing, cell cycle scoring, and visualizing expression values on the data.

Here we use a standard technique for the extraction of highly variable genes from the 10X genomics preprocessing software *CellRanger*. Typically between 1000 and 5000 genes are selected. Here, we extract the top 4000 most variable genes for further processing. If particular genes of importance are known, one could assess how many highly variable genes are necessary to include all, or the majority, of these.

Compute highly variable genes and visualize.

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', n_top_genes=2000)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))

In [ ]:
rcParams['figure.figsize']=(10,5)
sc.pl.highly_variable_genes(adata)

The plots show how the data was normalized to select highly variable genes irrespective of the mean expression of the genes. This is achieved by using the index of dispersion which divides by mean expression, and subsequently binning the data by mean expression and selecting the most variable genes within each bin.

Highly variable gene information is stored automatically in the `adata.var['highly_variable']` field. The dataset now contains:

* count data as layer 'counts' in adata
* log-scran normalized data in adata.X
* highly variable gene annotations in `adata.var['highly_variable']`


## Visualization

Visualizing scRNA-seq data is the process of projecting a high-dimensional matrix of cells and genes into a few coordinates such that every cell is meaningfully represented in a two-dimensional graph. However, the visualization of scRNA-seq data is an active area of research and each method defines 'meaningful' in its own way. 

Overall t-SNE visualizations have been very popular in the community, however the recent UMAP algorithm has been shown to better represent the topology of the data.

We will look at several visualizations to decide which visualization best represents the aspect of the data.

Compute the following embeddings: PCA, t-SNE, UMAP, diffusion map and force-directed graph. Please compute PCA first and compute nearest neighbors next. All other embeddings rely on this information. Visualize the embeddings and color by the total number of counts.

In [ ]:
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')

To determine the number of informative principal components, let us review the variance contribution of each component. 

In [ ]:
sc.pl.pca_variance_ratio(adata)

Using the elbow method, we select the first **XX** PCs as informative.

**Task:** Decide on the number of informative principal components.

In [ ]:
sc.pp.pca(adata, n_comps= ,#number of informative components  
          use_highly_variable=True, svd_solver='arpack')
sc.pp.neighbors(adata)
sc.tl.tsne(adata) #Note n_jobs works for MulticoreTSNE, but not regular implementation)
sc.tl.umap(adata)
sc.tl.diffmap(adata)
sc.tl.draw_graph(adata)

In [ ]:
rcParams['figure.figsize']=(20,10)
fig_ind=np.arange(231, 237)
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.6)

p10 = sc.pl.pca_scatter(adata, color='n_counts', ax=fig.add_subplot(fig_ind[0]), show=False)
p11 = sc.pl.tsne(adata, color='n_counts', ax=fig.add_subplot(fig_ind[1]), show=False)
p12 = sc.pl.umap(adata, color='n_counts', ax=fig.add_subplot(fig_ind[2]), show=False)
p13 = sc.pl.diffmap(adata, color='n_counts', components=['1,2'], ax=fig.add_subplot(fig_ind[3]),show=False)
p14 = sc.pl.diffmap(adata, color='n_counts', components=['1,3'], ax=fig.add_subplot(fig_ind[4]), show=False)
p15 = sc.pl.draw_graph(adata, color='n_counts', ax=fig.add_subplot(fig_ind[5]), show=False)

plt.show()

**PCA**:

* Unsurprisingly, the first principle component captures variation in count depth between cells, and is thus only marginally informative
* The plot shows a weak clustering of the data in two dimensions

**t-SNE**:

* Shows several distinct clusters with clear subcluster structure
* Connections between clusters are difficult to interpret visually

**UMAP**:

* Data points are spread out on the plot showing several clusters
* Connections between clusters also not visible (and not expected for PBMC data)

**Diffusion Maps**:

* Shows regions of higher density and a few disconnected cells in between
* Trajectories not visible (and not expected for PBMC data)
* Each diffusion component extracts heterogeneity in a different part of the data

**Graph**:

* Shows several clusters with substructure
* Lack of trajectories as expected

The strengths and weaknesses of the visualizations can readily be identified in the above plots. While t-SNE exaggerates differences, diffusion maps exaggerate transitions. Overall UMAP and force-directed graph drawings show the best compromise of the two aspects, however UMAP is much faster to compute. UMAP has furthermore been shown to more accurately display the structure in the data.

## Cell cycle scoring

Known sources of technical variation in the data have been investigated and corrected for (e.g. batch, count depth). A known source of biological variation that can explain the data is the cell cycle. Here, gene lists from [Macosko et al., Cell 161 (2015)](https://www.sciencedirect.com/science/article/pii/S0092867415005498) is used to score the cell cycle effect in the data and classify cells by cell cycle phase. The file can be found on the [scIB github repository](https://github.com/theislab/scib/tree/master/scIB/resources/).

Please note, that the gene list was generated for human HeLa cells.

In [ ]:
s_genes_file = data_dir + 's_genes_tirosh_hm.txt'
g2m_genes_file = data_dir + 'g2m_genes_tirosh_hm.txt'

In [ ]:
s_genes = pd.read_table(s_genes_file, header = None).values.flatten()
g2m_genes = pd.read_table(g2m_genes_file, header = None).values.flatten()

In [ ]:
s_genes_hvg = adata.var_names[np.in1d(adata.var_names, s_genes)]
g2m_genes_hvg = adata.var_names[np.in1d(adata.var_names, g2m_genes)]

Compute cell cycle score per batch.

In [ ]:
adata.obs['S_score']= np.zeros(adata.shape[0])
adata.obs['G2M_score'] = np.zeros(adata.shape[0])
adata.obs['phase'] = np.zeros(adata.shape[0])

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes_hvg, g2m_genes=g2m_genes_hvg)

In [ ]:
print(len(s_genes_hvg))
print(len(g2m_genes_hvg))

In [ ]:
adata.obs['phase'].value_counts()

**Task:** Check whether MKI67 is present in the dataset and visualise the cell cycle scores.

In [ ]:
#hint 'MKI67' has to be in the adata.var_names 

In [ ]:
rcParams['figure.figsize']=(5,5)
#plots

Apparently, all cells are assigned to the same proliferative cell cycle phase (G2M), but inspecting the interface marker MKI67 shows little evidence for proliferation. If the phase score looks suspicious, one has to adjust the threshold for assigning a certain phase. By default, a cell is G1, if both S and G2M score are negative. Otherwise, a cell is assigned to the phase where it has the highest score. Potentially, we would need to adapt the cutoffs of the classification.

At this point of the analysis, we have reached an important milestone as we finished the pre-processing and enter the downstream analysis part. **Ideally, we don't have to revisit this part again.**

Of note, before we save the data to file, we convert the gene expression matrix X to the sparse format to memory.

In [ ]:
import scipy.sparse as sparse

In [ ]:
adata.X = sparse.csr_matrix(adata.X)

In [ ]:
adata.write(data_dir + 'data_processed.h5ad')

**Comment:** End of third session.

# Downstream analysis

In [ ]:
adata = sc.read(data_dir + 'data_processed.h5ad')

## Clustering

Clustering is a central component of the scRNA-seq analysis pipeline. To understand the data, we must identify cell types and states present. The first step of doing so is clustering. Performing Modularity optimization by Louvain community detection on the k-nearest-neighbour graph of cells has become an established practice in scRNA-seq analysis. Thus, this is the method of choice in this tutorial as well.

Here, we perform clustering at two resolutions. Investigating several resolutions allows us to select a clustering that appears to capture the main clusters in the visualization and can provide a good baseline for further subclustering of the data to identify more specific substructure.

Clustering is performed on the highly variable gene data, dimensionality reduced by PCA, and embedded into a KNN graph. (see `sc.pp.pca()` and `sc.pp.neighbors()` functions used in the visualization section).

Compute a `louvain` clustering with two different resolutions (`0.5` and `1`). Compare the clusterings in a table and visualize the clustering in an embedding. Optional: Compute a clustering with the `leiden` algorithm. 

In [ ]:
# Perform clustering - using highly variable genes
sc.tl.louvain(adata, resolution=1.5, key_added='louvain_r1.5')
sc.tl.louvain(adata, resolution=0.5, key_added='louvain_r0.5')

In [ ]:
pd.crosstab(adata.obs['louvain_r0.5'], adata.obs['louvain_r1.5'])

In [ ]:
#Visualize the clustering and how this is reflected by different technical covariates
sc.pl.umap(adata, color=['louvain_r1.5', 'louvain_r0.5'], wspace=0.6)
sc.pl.umap(adata, color=['log_counts', 'mt_frac'])

## Marker genes and cluster annotation 

To annotate the clusters we obtained, we find genes that are up-regulated in the cluster compared to all other clusters (marker genes). This differential expression test is performed by a *Welch t-test with overestimated variance* to be conservative. This is the default in `scanpy`. The test is automatically performed on the `.raw` data set, which is uncorrected and contains all genes. All genes are taken into account, as any gene may be an informative marker.

As we are using the relevant mouse gut atlas from the literature in this case study, there is no other reference atlas which we use to annotate the cells by automated annotation. Thus, we do not use scmap or garnett here.

Compute the differential expression profile for each cluster with `rank_genes_groups` and visualize the results.

In [ ]:
#Calculate marker genes
sc.tl.rank_genes_groups(adata, groupby='louvain_r0.5', key_added='rank_genes_r0.5')

In [ ]:
#Plot marker genes
sc.pl.rank_genes_groups(adata, key='rank_genes_r0.5', fontsize=12)

**Tasks:** Calculate and visualise marker genes for the louvain clustering with resolution `1.5`.

In [ ]:
#Calculate marker genes


In [ ]:
#Plot marker genes


Here, we observe potentially characteristic gene expression patterns, but we also observe a considerable ribosomal proteins (*RPL* and *RPS*), which are part of the ribosomes. Thus, they are involved in mRNA translational processes. Usually, these genes are difficult to interpret. 

Furthermore, the score itself is not interpretable in terms of specificity and significance in the case of clustering, because the clusters were previously defined as a group of cells being different from the rest. Therefore, we compare a group that is a priori different to the rest and the resulting scores (or p-values) are inflated. Furthermore, the smaller a cluster is, the smaller is the observed score, unless a gene is very specific to the cluster. Typically, we may find marker genes in the gene lists of the `rank_genes_groups` test, but not all marker genes have a high expression level.

When it comes to cluster annotation, we usually have to tap into prior knowledge of the cell type. Depending on the data set, this may involve extensive literature search. In the case of brain cell types, we may refer to several studies and several web resources to extract marker gene sets. Alternative approaches such as `scmap` use annotated reference data to predict the cell type identity of new data, or train a classifier based on marker genes (e.g. `Garnett`).


In the case of PBMCs, we may refer to several studies and single-cell RNA-sequencing data analysis tutorials to extract marker gene sets. 
The following list is extracted from the Seurat tutorial on PBMCs.


|Marker Gene|Cell Type|
|---------|-------|
|IL7R|CD4 T cells|
|CD14, LYZ|CD14+ Monocytes|
|MS4A1|B cells|
|CD8A|CD8 T cells|
|FCGR3A, MS4A7|FCGR3A+ Monocytes|
|GNLY, NKG7|NK cells|
|FCER1A, CST3|Dendritic Cells|
|PPBP|Megakaryocytes|


Let us define a list of marker genes from literature.

In [ ]:
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']

**Tasks:** Annotate the clusters. 
Check briefly, if all marker genes are present in the dataset and visualise the marker genes in a UMAP (or another visualisation of your choice).
You can use auxiliary plots like `matrixplot`, `dotplot`, `heatmap` or `violin` plots or coloring an embedding (e.g. UMAP, t-SNE, FA) by the marker genes.

Let us check if the marker genes are expressed in our dataset.

In [ ]:
np.in1d(marker_genes, adata.var_names)

In [ ]:
#plots

In [ ]:
sc.pl.dotplot(adata=, 
              var_names =,
              groupby=, 
              use_raw=False)

In [ ]:
sc.pl.heatmap(adata=, var_names=, 
              figsize=(5,10),
              groupby=, 
              use_raw=False, vmin=0)

In [ ]:
sc.pl.matrixplot(adata=, var_names=,
                 groupby=, 
                 use_raw=False, vmin=0)

In [ ]:
sc.pl.stacked_violin(adata = ,var_names = , groupby=, 
                     use_raw=False)

Annotate clusters and create a new covariate.


|Cluster ID|Marker Gene|Cell Type|
|---------|-------|-------|
||IL7R|CD4 T cells|
||CD14, LYZ|CD14+ Monocytes|
||MS4A1|B cells|
||CD8A|CD8 T cells|
||FCGR3A, MS4A7|FCGR3A+ Monocytes|
||GNLY, NKG7|NK cells|
||FCER1A, CST3|Dendritic Cells|
||PPBP|Megakaryocytes|

Use the `pandas` data frame functionality to rename your clusters and visualize your annotation.

In [ ]:
adata.obs['annotated'] = adata.obs['louvain_r1.5'].cat.add_categories(['CD4 T cells', 
                        'CD14+ Monocytes', 'B cells', 'CD8 T cells', 
                        'FCGR3A+ Monocytes', 'NK cells', 'Dendritic cells', 'Megakaryocytes'])

adata.obs['annotated'][np.in1d(adata.obs['annotated'], [ #add cluster name here (as string)    
                                                            ])] = 'CD4 T cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], [])] = 'CD14+ Monocytes'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], [])] = 'B cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], [])] = 'CD8 T cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], [])] = 'FCGR3A+ Monocytes'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], [])] = 'NK cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], [])] = 'Dendritic cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], [])] = 'Megakaryocytes'

#remove unused categories from annotation
adata.obs['annotated'] = adata.obs['annotated'].cat.remove_unused_categories()

In [ ]:
adata.obs['annotated'].value_counts()

**Task:** Visualise your annotation on a UMAP as well as in a `matrixplot`, `dotplot`, `heatmap` or `violin` plots.

In [ ]:
sc.pl.umap(adata, color='annotated', legend_loc='on data', title='', frameon=False)
sc.pl.umap(adata, color='annotated',  title='', frameon=True)

In [ ]:
sc.pl.dotplot(adata=, 
              var_names =,
              groupby=, 
              use_raw=False)

In [ ]:
sc.pl.heatmap(adata=, var_names=, 
              figsize=(5,10),
              groupby=, 
              use_raw=False, vmin=0)

In [ ]:
sc.pl.matrixplot(adata = , 
                 var_names = , 
                 groupby= , 
                 use_raw=False, vmin=0)

In [ ]:
sc.pl.stacked_violin(adata=   , 
                     var_names=   , 
                     groupby=    , 
                     use_raw=False)

### Inspect subpopulations of B cells

Let us determine the differences in the B cell clusters by differential expression. Subcluster the B cells first.

In [ ]:
sc.tl.louvain(adata, resolution=0.2, restrict_to = ['annotated',['B cells']], key_added='louvain_R')

In [ ]:
rcParams['figure.figsize']= (5,5)
sc.pl.umap(adata, color='louvain_R')

In [ ]:
sc.tl.rank_genes_groups(adata = adata, groupby='louvain_R', groups= ['B cells,1'], reference='B cells,0', rankby_abs=True)

In [ ]:
rcParams['figure.figsize']=(10,5)
sc.pl.rank_genes_groups(adata, size=10, n_genes=40)

In [ ]:
sc.pl.rank_genes_groups_violin(adata, groups='B cells,1', n_genes=10, use_raw=False)

Here, the B cell populations differ in B cell activation markers such as TCL1A and IL4R. Interestingly, the `B cells,1` population is almost exclusively positive for the *IGKC* gene. The *IGKC* gene was found to be predictive for cancer prognosis.
Potentially, `B cells,1` are activated B cells and `B cells,0` are resting, but we have to look at more markers to make the distinction. 

### Compute a PAGA for PBMCs

We aim to find relations between the respective cell types based on the knn graph with partition-based graph abstraction (PAGA). 

Compute PAGA on the cluster annotation and plot the graph (note: use the plot function `paga_compare`).

In [ ]:
sc.tl.paga(adata = adata, groups='annotated')

In [ ]:
rcParams['figure.figsize']=(7,7)
sc.pl.paga_compare(adata = adata, basis='umap', frameon=True)

## Save annotated data to file

At this point, we have finished the data annotation. This represents another milestone in the data analysis of single cell data. Once the annotation is finished, we won't have to touch this part of the analysis again.   

In [ ]:
adata.write(data_dir + 'data_processed.h5ad')

## Pseudotime on Monocytes

In this section, we want to explore a potential transition of CD14+ and FCGR3A+ Monocytes.

In [ ]:
adata = sc.read(data_dir + 'data_processed.h5ad')

Select the monocytes.

In [ ]:
adata_mono = adata[np.in1d(adata.obs['annotated'], 
                           ['CD14+ Monocytes', 'FCGR3A+ Monocytes'])].copy()

In [ ]:
adata_mono

Compute a diffusion pseudotime (DPT). Note: Fix a root cell as `adata.uns['iroot']` first. Visualize the pseudotime on the embedding. Compute the differentially expressed genes between the two subgroups and visualize the expression of 20 top differentially expressed genes along pseudotime.

In [ ]:
sc.tl.pca(adata_mono, svd_solver='arpack')
sc.pp.neighbors(adata_mono)

Convert UMAP indices to arrays.

In [ ]:
umap_0 = [term[0] for term in adata_mono.obsm['X_umap']]
umap_1 = [term[1] for term in adata_mono.obsm['X_umap']]

Set root cell to the cell with the smallest value in the first UMAP component and compute DPT.

In [ ]:
adata_mono.uns['iroot'] = np.flatnonzero(umap_0== max(umap_0))[0]
sc.tl.dpt(adata = adata_mono)

Visualise DPT on a UMAP and on a diffusion map.

In [ ]:
rcParams['figure.figsize']=(7,7)
sc.pl.umap(adata_mono, color=['dpt_pseudotime', 'annotated'])

In [ ]:
rcParams['figure.figsize']=(7,7)
sc.pl.diffmap(adata_mono, color=['dpt_pseudotime', 'annotated'], components=['1,2'])
sc.pl.diffmap(adata_mono, color=['dpt_pseudotime', 'annotated'], components=['1,3'])

Run a differential test on the two groups of monocytes in order to determine characteristic genes.  

In [ ]:
sc.tl.rank_genes_groups(adata_mono, groupby='annotated', 
                        groups= ['FCGR3A+ Monocytes'], reference='CD14+ Monocytes', rankby_abs=True)

In [ ]:
rcParams['figure.figsize']=(10,5)
sc.pl.rank_genes_groups(adata_mono, size=10, n_genes=30)

In [ ]:
rcParams['figure.figsize']= (15,5)
sc.pl.rank_genes_groups_violin(adata_mono, use_raw=False)

In [ ]:
mono_genes = [idx[1][0] for idx in enumerate(adata_mono.uns['rank_genes_groups']['names'])]


In order to visualise the gene expression along pseudotime, we have to compute PAGA for the two groups of monocytes.

In [ ]:
sc.tl.paga(adata_mono, groups='annotated')

Modify the format of the data matrix, because `paga_path` takes only dense matrices (in this `scanpy` version).

In [ ]:
adata_mono.X = adata_mono.X.todense()

In [ ]:
rcParams['figure.figsize']=(20,10)
sc.pl.paga_path(adata_mono, nodes=['FCGR3A+ Monocytes','CD14+ Monocytes'], 
                keys=mono_genes[:25],n_avg=10, use_raw=False, save='_monocyte_transition.pdf')

**Comment:** End of fourth session.